# Aave Proposal Pivotality

This article is part of a series of reports that aims to determine the impact of large token holders (whales) on DAO governance.

We take a look at 10 recent proposals on Aave Governance, including the first Butter Aave Elections, and compare the voter turnout and the "pivotality" (a measure of whale influence) of each of them.

In [1]:
# sets up the pynb environment
import os
import sys

from IPython.display import HTML
import pandas as pd

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from stages.data_processing.statistics import (
    get_number_of_whales_to_all_voters_ratio,
    get_score_comparisons,
)

In [2]:
all_organization_proposals = pd.read_csv(
    "../plutocracy_data/full_report/Aave_report.csv.gzip",
    engine="c",
    low_memory=False,
    compression='gzip',
)
all_organization_proposals_filtered = pd.read_csv(
    "../plutocracy_data/full_report/Aave_report_filtered.csv.gzip",
    engine="c",
    low_memory=False,
    compression='gzip',
)


In [3]:
def to_organization_map(flat_organization_dataframe: pd.DataFrame):
    return {
        organization_name: proposal_df
        for organization_name, proposal_df in [
            (str(space_name), space_proposals)
            for space_name, space_proposals in flat_organization_dataframe.groupby(
                "proposal_space_name"
            )
        ]
    }

plutocracy_report_data = to_organization_map(all_organization_proposals)
plutocracy_report_data_filtered = to_organization_map(all_organization_proposals_filtered)


In [4]:
pd.set_option("display.max_rows", int(1e3))
score_differences = get_score_comparisons(
    plutocracy_report_data, plutocracy_report_data_filtered
)


In [109]:
score_differences_dfs: dict[str, pd.DataFrame] = dict()

initial_series_data = {
    organization: 0
    for organization in plutocracy_report_data.keys()
}
changed_outcome_proportions = pd.Series(initial_series_data, name="changed outcomes %")

for score_difference in score_differences:
    for organization, data in score_difference.items():
        data: dict[str, list] = data
        items = data.items()
        score_differences_dfs[organization] = pd.DataFrame(
            [score_data for _, score_data in items],
            index=pd.Index(
                ([id for id, _ in items]), name="Proposal"
            ),
            columns=[
                "Title",
                "Date Started",
                "Date Ended",
                "score_differences",
                "whale_vp_proportion",
                "total_vp",
                "outcome_changed",
                "Result",
                "Result w/o Whales"
            ],
        ).astype({"total_vp": "float64"}, copy=False
        ).sort_values(["whale_vp_proportion","total_vp"], ascending=False)

        changed_outcome_proportions[organization] = score_differences_dfs[organization]["outcome_changed"].value_counts(normalize=True)[True]

        score_differences_dfs[organization].drop("outcome_changed", axis=1, inplace=True)

        score_differences_dfs[organization]["Date Started"] = pd.to_datetime(score_differences_dfs[organization]["Date Started"], utc=True, unit="s").dt.strftime("%d/%m/%y")
        score_differences_dfs[organization]["Date Ended"] = pd.to_datetime(score_differences_dfs[organization]["Date Ended"], utc=True, unit="s").dt.strftime("%d/%m/%y")

        space_id = plutocracy_report_data[organization].iloc[0]["proposal_space_id"]

        score_differences_dfs[organization]["total_vp"] = score_differences_dfs[
            organization
        ]["total_vp"].apply("{:.9f}".format)

        score_differences_dfs[organization]["Title"] = score_differences_dfs[organization].apply(
            lambda row: f'<a href=http://snapshot.org/#/{space_id}/proposal/{row.name} rel="noopener noreferrer" target="_blank">{row["Title"]}</a>',
            axis=1
        )
        score_differences_dfs[organization].style.format({"whale_vp_proportion": "{:.2%}".format})
changed_outcome_proportions = changed_outcome_proportions.apply(
    lambda proportion: "{:.2%}".format(proportion)
)

In [6]:
voting_ratios = get_number_of_whales_to_all_voters_ratio(
    plutocracy_report_data, plutocracy_report_data_filtered
)


## Synthesis

Show the percentage of proposals whose outcome changes if whales did not vote (i.e. *whale pivotality*).

In [110]:
dao_overview = pd.DataFrame(
    [list(result.items())[0][1] for result in voting_ratios],
    columns=[
        "# of whales",
        "all voters",
    ],
)
dao_overview.set_index(
    pd.Index([list(result.items())[0][0] for result in voting_ratios], name="DAO"),
    inplace=True
)

dao_overview.insert(2, changed_outcome_proportions.name, changed_outcome_proportions)
dao_overview

,# of whales,all voters,changed outcomes %
DAO,,,
Aave,651,16006,11.11%


---

## Case Studies

### Filtered Proposals Analysis

In [111]:
HTML(score_differences_dfs["Aave"].rename(lambda index: score_differences_dfs["Aave"].loc[index]["Title"]).drop(["score_differences", "total_vp", "whale_vp_proportion", "Title"], axis=1).to_html(render_links=True, escape=False))

,Date Started,Date Ended,Result,Result w/o Whales
Proposal,,,,
[TEMP CHECK] Incentivized Delegate Campaign (3-month),03/04/23,10/04/23,TokenLogic,Blockworks Research
[ARFC] Aave V1 Offboarding Plan,13/04/23,16/04/23,YAE,YAE
[TEMP CHECK] Aave V3 Deployment on zkSync Era Mainnet,12/04/23,15/04/23,YAE,YAE
[TEMP CHECK] - Launch Aave V3 on Starknet,03/04/23,10/04/23,YEA,YEA
"[ARC] E-Mode Parameter Changes for V3 Avalanche, Optimism, Polygon, and Arbitrum",05/04/23,12/04/23,Option 3,Option 3
[TEMP CHECK] - Whitelist Stargate for V3 Portals,13/04/23,18/04/23,YAE,YAE
[TEMP CHECK] - GHO Facilitator Onboarding Process and Application,05/04/23,08/04/23,YAE,YAE
[ARFC] Align AAVE Risk Parameters on Aave V3 Ethereum Market with Aave V2,12/04/23,15/04/23,YAE,YAE
Risk Stewards Phase 1: CapsPlusSteward,12/04/23,17/04/23,YES 100% max increase per 5 days,YES 100% max increase per 5 days


### Proportion of Outcomes Changed

In [112]:
print(f"{changed_outcome_proportions['Aave']} of Aave's proposal outcomes change after filtering out whale voting power.")

11.11% of Aave's proposal outcomes change after filtering out whale voting power.


---

### Proposal Analysis

#### [TEMP CHECK] Incentivized Delegate Campaign (3-month)

Aka the first Butter'd Aave Election.

**Proposal ID**: [0xf6da417](https://snapshot.org/#/aave.eth/proposal/0xf6da417983653f7e7f5560a8e16ad77d5e25533bba9157abfd222347b40d09d0)

**Start Date**: 03/04/23
**End Date**: 10/04/23

**Voting System**: Weighted Voting

**Voting Strategies**:
- aAAVE
- AAVE + stkAAVE
- Matic AAVE + amAAVE
- AAVE in stkBPT (Balancer Pool Token)



Aave votes to select a delegate who will receive a $15k grant (paid in AAVE) from Aave Grants after a three-month term as a delegate, a delegate election. Thirteen delegates expressed interest in running this election which ran for 7 days. TokenLogic ultimately won this election, however, if we remove whale voting power from this proposal we find that Blockworks Research would've been selected by the Aave community instead.

In [102]:
propsal_choices = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0xf6da417983653f7e7f5560a8e16ad77d5e25533bba9157abfd222347b40d09d0'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Aave"].index.to_series().apply(
    lambda s: "0xf6da417" in s,
)
proposal_score_differences = score_differences_dfs["Aave"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0xf6da417983653f7e7f5560a8e16ad77d5e25533bba9157abfd222347b40d09d0'].iloc[0]['proposal_scores']


result = pd.DataFrame(
    {choice: [score, score_diff, score - score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Whale-only Scores", "Non-whale Scores"],
)

result

0.24937532803856208


,Blockworks Research,ConsenSys,Curia,DAOStewards,Fire Eyes,Flipside Crypto,FranklinDAO,OnChainCoop,Oxytocin,Saludiego201.eth,StableLab,TokenLogic,Wallfacer Labs
Scores,87232.635594,17620.565593,12742.462405,434.508563,104834.493934,90345.343934,66127.639535,409.610873,30504.177435,437.382397,146246.026838,185171.775039,435.859890
Whale-only Scores,86487.989188,16950.926291,12712.705752,407.073261,104802.547664,90318.258696,65886.207100,391.862324,30491.294636,393.737487,146194.584615,185153.921653,420.744217
Non-whale Scores,744.646406,669.639302,29.756653,27.435302,31.946270,27.085238,241.432435,17.748550,12.882799,43.644910,51.442223,17.853386,15.115673


---

#### [ARFC] Aave V1 Offboarding Plan

**Proposal ID**: [0x569a161](https://snapshot.org/#/aave.eth/proposal/0x569a161ff9e3684ad5fd767b72639a389ed6f58a838a3ab1b57ccd309058e026)

**Start Date**: 13/04/23
**End Date**: 16/04/23

**Voting System**: Single Choice Voting


**Voting Strategies**:
- aAAVE
- AAVE + stkAAVE
- Matic AAVE + amAAVE
- AAVE in stkBPT (Balancer Pool Token)

This proposal if passed will implement a new rate strategy for all Aave V1 assets to disincentive users from using V1, and move their assets to V2 or V3 instead. The overwhelming majority of voters voted for this proposal and filtering out whales did not change its outcome.

In [97]:
propsal_choices = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0x569a161ff9e3684ad5fd767b72639a389ed6f58a838a3ab1b57ccd309058e026'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Aave"].index.to_series().apply(
    lambda s: "0x569a161" in s,
)
proposal_score_differences = score_differences_dfs["Aave"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0x569a161ff9e3684ad5fd767b72639a389ed6f58a838a3ab1b57ccd309058e026'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff, score - score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Whale-only Scores", "Non-whale Scores"],
)

,YAE,NAY,ABSTAIN
Scores,427930.950274,75.491451,7.338264
Whale-only Scores,424165.546300,66.034293,1.199240
Non-whale Scores,3765.403974,9.457158,6.139025


---

#### [TEMP CHECK] Aave V3 Deployment on zkSync Era Mainnet

**Proposal ID**: [0x46cf72d](https://snapshot.org/#/aave.eth/proposal/0x46cf72da892eb216edc1b7fe2f24f7491d8c37344b2b1f67632fa6950be034f7)

**Start Date**: 12/04/23
**End Date**: 15/04/23

**Voting System**: Single Choice Voting


**Voting Strategies**:
- aAAVE
- AAVE + stkAAVE
- Matic AAVE + amAAVE
- AAVE in stkBPT (Balancer Pool Token)



A temperature check to gauge the community's sentiments on deploying Aave V3 on [zkSync Era](https://zksync.io/) Mainnet. The main purpose of this proposal is to encourage ecosystem growth. This proposal's outcome also was not changed after filtering out whales.

In [98]:
propsal_choices = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0x46cf72da892eb216edc1b7fe2f24f7491d8c37344b2b1f67632fa6950be034f7'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Aave"].index.to_series().apply(
    lambda s: "0x46cf72d" in s,
)
proposal_score_differences = score_differences_dfs["Aave"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0x46cf72da892eb216edc1b7fe2f24f7491d8c37344b2b1f67632fa6950be034f7'].iloc[0]['proposal_scores']


result = pd.DataFrame(
    {choice: [score, score_diff, score - score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Whale-only Scores", "Non-whale Scores"],
)

print()
result

,YAE,NAY,ABSTAIN
Scores,577937.331903,1133.242375,134.422288
Whale-only Scores,529876.707255,1.350356,1.585889
Non-whale Scores,48060.624648,1131.892020,132.836399


---

#### [TEMP CHECK] - Launch Aave V3 on Starknet

**Proposal ID**: [0xde90f4d](https://snapshot.org/#/aave.eth/proposal/0xde90f4d29c4c5dd55cba023b29bdde8c49e37f70c3398668a78e2f173a61b4e6)

**Start Date**: 03/04/23
**End Date**: 10/04/23

**Voting System**: Single Choice Voting


**Voting Strategies**:
- aAAVE
- AAVE + stkAAVE
- Matic AAVE + amAAVE
- AAVE in stkBPT (Balancer Pool Token)



This is a proposal to gauge the Aave community's sentiments on going on with phase 2 of the Aave's integration on Starknet. This phase will involve the deployment of Aave V3 on Starknet. Like the last 3 proposals detailed in this report, its outcome did not change after filtering out whales.

In [100]:
propsal_choices = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0xde90f4d29c4c5dd55cba023b29bdde8c49e37f70c3398668a78e2f173a61b4e6'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Aave"].index.to_series().apply(
    lambda s: "0xde90f4d" in s,
)
proposal_score_differences = score_differences_dfs["Aave"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0xde90f4d29c4c5dd55cba023b29bdde8c49e37f70c3398668a78e2f173a61b4e6'].iloc[0]['proposal_scores']
pd.DataFrame(
    {choice: [score, score_diff, score - score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Whale-only Scores", "Non-whale Scores"],
)

,YEA,NOP,ABSTAIN
Scores,664848.446683,21.627573,136.417016
Whale-only Scores,407757.376934,4.772256,120.558800
Non-whale Scores,257091.069749,16.855317,15.858216


---

#### [ARC] E-Mode Parameter Changes for V3 Avalanche, Optimism, Polygon, and Arbitrum

**Proposal ID**: [0x84deca8](https://snapshot.org/#/aave.eth/proposal/0x84deca82139320b2570f04211b249e37b8a7602b4a0ed70e6fa772c9f6e94550)

**Start Date**: 05/04/23
**End Date**: 11/04/23

**Voting System**: Single Choice Voting


**Voting Strategies**:
- aAAVE
- AAVE + stkAAVE
- Matic AAVE + amAAVE
- AAVE in stkBPT (Balancer Pool Token)



In [101]:
propsal_choices = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0x84deca82139320b2570f04211b249e37b8a7602b4a0ed70e6fa772c9f6e94550'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Aave"].index.to_series().apply(
    lambda s: "0x84deca8" in s,
)
proposal_score_differences = score_differences_dfs["Aave"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0x84deca82139320b2570f04211b249e37b8a7602b4a0ed70e6fa772c9f6e94550'].iloc[0]['proposal_scores']
pd.DataFrame(
    {choice: [score, score_diff, score - score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Whale-only Scores", "Non-whale Scores"],
)

,Option 1,Option 2,Option 3,Option 4,Abstain,Nay
Scores,126545.660433,166.034309,532246.554661,140.712281,489.850798,2.067035
Whale-only Scores,84143.973524,83.499083,291145.238949,107.559375,416.456718,1.017788
Non-whale Scores,42401.686910,82.535226,241101.315713,33.152905,73.394080,1.049248
